Data Wrangling Notebook for HMachado Equus Data
<br />
Neeka Sewnath
<br />
nsewnath@ufl.edu

In [4]:
import pandas as pd
import uuid

Import Horse Data

In [5]:
# Importing horse data from a comma delimited file
horseData = pd.read_csv("../Original_Data/Horse data_Helena.csv")

Preliminary data cleaning 

In [6]:
# Cleaning decimalLatitude by removing S and making value negative
for ind in horseData.index:
    a = horseData['decimalLatitude'][ind]
    b = str(a)
    c=b[:-1]
    c="-"+c
    horseData['decimalLatitude'][ind] = c

# Cleaning decimalLongitude by removing W and making value negative
for ind in horseData.index:
    a = horseData['decimalLongitude'][ind]
    b = str(a)
    c=b[:-1]
    c="-"+c
    horseData['decimalLongitude'][ind] = c

na_long = horseData['decimalLatitude']=="-na"
na_lat = horseData['decimalLongitude']=="-na"
horseData['decimalLatitude'][(na_long == True)]= ""
horseData['decimalLongitude'][(na_lat == True)]= ""


<ipython-input-6-0c39a2199422>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  horseData['decimalLatitude'][ind] = c
<ipython-input-6-0c39a2199422>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  horseData['decimalLongitude'][ind] = c
<ipython-input-6-0c39a2199422>:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  horseData['decimalLatitude'][(na_long == True)]= ""
<ipython-input-6-0c39a2199422>:20: SettingWithCopyWarning: 
A value 

In [7]:
# Combining SpecimenType to MeasurementType Columns
horseData['test'] = horseData['specimenType'].str.cat(horseData['measurementType'])

# Standardization from fossil to FossilSpecimen
horseData.loc[horseData['basisOfRecord'] == 'fossil', 'basisOfRecord'] = 'FossilSpecimen'
#horseData['basisOfRecord']

# Measurement unit from millimeters to mm
horseData.loc[horseData['measurementUnit'] == 'millimeters', 'measurementUnit'] = 'mm'
#horseData['measurementUnit']

In [31]:
# Rearrange columns so that template columns are first, followed by measurement values

# Create column list
cols = horseData.columns.tolist()

# Specify desired columns
cols = ['institutionCode',
        'collectionCode',
        'specimenID',
        'side',
        'test',
        'scientificName',
        'decimalLatitude',
        'decimalLongitude',
        'sitename',
        'verbatimLocality',
        'basisOfRecord',
        'measurementValue',
        'measurementUnit',
        'lithostratigraphicTerms',
        'formation',
        'member',
        'references']

#Subset dataframe
horseData = horseData[cols]

In [32]:
# Matching template and column terms

#Renaming columns 
horseData = horseData.rename(columns = {'specimenID':'catalogNumber',
                                        'specimenType':'skeletalElement',
                                        'side':'measurementSide',
                                        'sitename':'locality',
                                        'test': 'measurementType',
                                        'reference':'measurementMethod'})

In [33]:
#Sectioning off ingested data

#horseData = horseData[horseData[""]]

In [34]:
#create materialSampleID which is a UUID for each measurement
horseData=horseData.assign(materialSampleID = '')
horseData['materialSampleID'] = [uuid.uuid4().hex for _ in range(len(horseData.index))]

In [35]:
# Create GEOME required columns 
horseData["scientificName"]= horseData["scientificName"].fillna("Unknown")
horseData=horseData.assign(eventID = horseData["materialSampleID"])
horseData=horseData.assign(measurementMethod = "unknown")
horseData=horseData.assign(country = "unknown")
horseData=horseData.assign(yearCollected = "unknown")
horseData=horseData.assign(samplingProtocol = "unknown")

Create diagnosticID

In [36]:
#create diagnosticID which is a UUID for each measurement
horseData=horseData.assign(diagnosticID = '')
horseData['diagnosticID'] = [uuid.uuid4() for _ in range(len(horseData.index))]

Write file to csv

In [37]:
#Writing the final dataframe as csv file
horseData.to_csv('../Mapped_Data/FuTRES_Equus_HMachado_Americas_paleo.csv')

In [ ]:
#End Notes
#measurementUnit still has empty columns 
#the word "cast" in basisOfRecord
#check what traits are ready to be ingested